# Introduction
The purpose of this tutorial is to walk you through how to executing the code to solve the shock tube using 3 different schemes: Roe, AUSM, and AUSM+.
Additionally, this tutorial gives a brief background on where the equations come for Roe, AUSM, and AUSM+ come from as links to tutorials and references. 
There are a lot of good comments codes. The reader is encouraged to execute the code, read the paper, and then read the code to understand how the equations from the paper are used. 


## Shock Tube Background
A shock tube is a tube that has high pressure in one region and low pressure in the other separated by a diaphram such as a rupture disk. The signficance of this problem is that it can be solved analytically which makes it great for testing numerical schemes. It can also be used to test fast response pressure probes and thermocouples. When the diaphragm bursts, there is a sudden expansion wave that accelerates the flow from high pressure to low pressure indicated in region II. The angle of this expansion wave increase over time \(you will see this later\). This means the flow is speeding up. There's a also a flat region where nothing is happening - this is the contact discontiunity. In this region Velocity, Pressure, Temperature, and Density are all constant. After the contact discontinuity, there are two shocks to bring the pressure down to the low pressure region.


<img src="docs/diagram.jpg" style="width:800px;"> </br>
<img src="docs/solution.jpg" style="width:800px;">




## Governing Equations
The equation below are the governing equation for a 1D shock tube problem. The $U$ vector contains the quantities of interest added to a flux vector $F(U)$. In the code the flux vector maybe denoted as the variable $q$. 

$$
\frac{\partial}{\partial t} \begin{bmatrix} 
\rho \\
\rho u \\ 
rho E
\end{bmatrix} 
+
\frac{\partial}{\partial x} \begin{bmatrix}
\rho u \\
\rho u^2 + P \\
(rhoE + P) * u
\end{bmatrix} = 0
$$

$U$ vector consits of the quantities changing with respect to time. 
$$
U = 
\frac{\partial}{\partial t} \begin{bmatrix} 
\rho \\
\rho u \\ 
rho E
\end{bmatrix} 
$$ 

The Flux vector $F(U)$ is written as a function of $U$ this way we can define it as a function in the code and compute it at the $U_{i+1}$ point for example. 
$$
F(U) = \begin{bmatrix}
\rho u \\
\rho u^2 + P \\
(rhoE + P) * u
\end{bmatrix}
$$

### Useful Equations 
Initial speed of sound
$$
a_0 = \sqrt(\gamma * P_0 / \rho )
$$

$`u_0`$ is a vector defining the initial velocity in the system
$$
dt = CFL * dx / max(|u_0| + a_0)
$$

Where 
$$
P = \rho*(\gamma - 1)(E - 0.5u^2)
$$


and $`\gamma = 1.4`$ 

Internal Energy is 
$$
e = c_v * T 
$$

$$
c_v = \frac{k}{\mu} * \frac{1}{\gamma-1}
$$

Viscosity $`\mu`$ and thermal conductivity $`k`$ for air needs to be looked up. Units of $`c_v`$ is J/(Kg K)

> Note:
> Solution must be stopped before the wave hits the boundary

# Roe Scheme
The code for using the [Roe Scheme](https://en.wikipedia.org/wiki/Roe_solver) to solve the shock tube comes from the tutorial video below. Roe scheme applies the chain rule to the flux vector. This creates a jacobian matrix $A(U)$. This matrix can then be represented as a matrix consisting of eigenvectors and eigenvalues $V\Lambda V^-1$ and used to compute the Roe Flux. 

For more detailed information, please see the comments in the code `Analytical-Roe.py` and the tutorial below. 

<div align="left">
      <a href="https://www.youtube.com/watch?v=F_PsFHvt8IU">
         <img src="https://img.youtube.com/vi/F_PsFHvt8IU/0.jpg" style="width:500px;">
      </a>
</div>

> Reference 
> [Roe Scheme Paper](references/Roe%20Scheme.pdf)

## Prerequisites 
Run the code below to install all the python prerequisites needed for the rest of the tutorial

In [1]:
!pip install -U moviepy
from IPython.display import HTML, Video
import os
from moviepy.editor import *
import moviepy
import sys 
def create_movie(image_folder:str):
    fps=12

    image_files = [os.path.join(image_folder,img)
                for img in os.listdir(image_folder)
                if img.endswith(".png")]
    clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
    clip.write_videofile(f'{image_folder}.mp4')
    clip.ipython_display()


The code below demonstrates how to execute the Roe Scheme from the tutorial. 

In [2]:
# To run the code
!rm roe_scheme_results.mp4
!python Analytical-Roe.py
create_movie('roe_scheme_results')
Video("roe_scheme_results.mp4",width=800)


'rm' is not recognized as an internal or external command,
operable program or batch file.


Configuration 1, Sod's Problem
Moviepy - Building video roe_scheme_results.mp4.
Moviepy - Writing video roe_scheme_results.mp4



Moviepy - Done !
Moviepy - video ready roe_scheme_results.mp4
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


# Ausm Scheme

AUSM is an upwind finite volume scheme that provides a way to calculate the flux in between the nodes \(i+1/2\). This lets us integrate the flux in and out of each cell boundary. Schemes such as Roe and TVD are accuracte but require more operations to achieve this accuracy, additionally the calculation of eigenvalues makes the scheme complex.

The goal of AUSM is to provide a simplified method of flux vector splitting that reduces numerical diffusion. 

Below are some except equations from the paper that are used in the code.For more detailed information, please see the comments in the code `ausm/analytical-ausm.py` 

<p align="center">
    <img src="ausm/domain_new.png" alt="Domain" width="500px"/>
</p>

The equation below is the calculation of Flux at 1/2 

<p align="center">
    <img src="ausm/F_half.png" alt="Flux Half" width="500px"/>
</p>

Mach 1/2 is used in the Flux 1/2 equation. Below is how to compute.

<p align="center">
    <img src="ausm/mach_half.png" alt="Mach Half" width="500px"/>
</p>

Either definition of Pressure at the halfway points is fine. 

<p align="center">
    <img src="ausm/pressure_pos_neg.png" alt="Pressure Positive Negative" width="500px"/>
</p>

> Reference:
> M.-S. Liou and C. J. Steffen Jr., A new flux splitting scheme, J. Comput. Phys. 107, 23 (1993). First order scheme.
> [AUSM](references/19910013501.pdf)


In [3]:
# Run the ausm scheme
!rm ausm_results.mp4
!python ausm/analytical-ausm.py  
create_movie('ausm_results')
Video("ausm_results.mp4",width=800)

'rm' is not recognized as an internal or external command,
operable program or batch file.


Moviepy - Building video ausm_results.mp4.
Moviepy - Writing video ausm_results.mp4



Moviepy - Done !
Moviepy - video ready ausm_results.mp4
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


# AUSM+ Scheme
AUSM+ addresses some deficiencies in the AUSM scheme and improve upon the following: *This list comes from the reference. Please read the paper, the code specifically refers to equations in the paper*

1. Exact resolution of 1D contact and shock discontinuities
2. Positivity preserving density (no negative density)
3. Elimates the [carbuncle phenomenon](https://blogs.sw.siemens.com/simcenter/mitigating-the-carbuncle-effect-for-hypersonic-cfd-simulations/)
4. Eliminates oscillations with slow moving shocks
5. Simplier algorithm
6. Extension to other hyperbolic systems

[AUSM+ Part I Paper](references/Sequel%20to%20AUSM%20Part%20I_AUSM%2B%20JCP129_Liou(1).pdf)

In [4]:
# Run the ausm scheme
!rm ausm+_results.mp4
!python ausm+/main_ausm_plus.py
create_movie('ausm+_results')
Video("ausm+_results.mp4",width=800)

'rm' is not recognized as an internal or external command,
operable program or batch file.


Moviepy - Building video ausm+_results.mp4.
Moviepy - Writing video ausm+_results.mp4



Moviepy - Done !
Moviepy - video ready ausm+_results.mp4
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


# AUSM+ Higher Order
The code presented in `ausm_higher_order/main_ausm_plus_higher_order.py` presents a small correction to the ausm paper's equation 65.

<img src="docs/ausm+_eqn65.png" width="600px">



> Reference:
> Liou, M. S. (1996). A sequel to ausm: Ausm+. Journal of computational Physics, 129(2), 364-382.
> [AUSM+ Part I](references/Sequel%20to%20AUSM%20Part%20II_AUSM%2B-up%20JCP214_Liou(1).pdf)


## Correction
Special thanks to Dr. Kenji Miki for discovering this correction.

```python
delta_half1 = np.zeros((3,q.shape[1])) # delta+1/2
for i in range(0,q.shape[1]-1):
    delta_half1[:,i] = q[:,i+1] - q[:,i] 

W[:,0] = q[:,0]
W[:,1] = q[:,1]
F_half[:,0] = flux_ausm_plus(W,gamma)[:,0]
for i in range(1,q.shape[1]-2):
    r_1 = delta_half1[:,i] / delta_half1[:,i-1] # Eqn 66 r_j
    r_2 = delta_half1[:,i] / delta_half1[:,i+1] # Eqn 66 r_j+1

    r_1 = np.nan_to_num(r_1,nan=0) 
    r_2 = np.nan_to_num(r_2,nan=0)
    if np.any(r_1<0): 
        W[:,0] = q[:,i]    
    else:
        W[:,0] = q[:,i] + 0.5 * minmod(r_1) * ( q[:,i] - q[:,i-1] ) # W_L

    if np.any(r_2<0): 
        W[:,1] = q[:,i+1]  
    else:        
        W[:,1] = q[:,i+1] - 0.5 * minmod(r_2) * ( q[:,i+2] - q[:,i+1] )# W_R <-- The correction is in the minmod 
    
    F_half[:,i] = flux_ausm_plus(W,gamma)[:,0] # Take the states and passes it to ausm plus
```


In [5]:
# Example of the code
!rm ausm+_higher_order_results.mp4
!python ausm_higher_order/main_ausm_plus_higher_order.py
create_movie('ausm+_higher_order_results')
Video("ausm+_higher_order_results.mp4",width=800)

'rm' is not recognized as an internal or external command,
operable program or batch file.
c:\Gitlab\shock-tube\ausm_higher_order\main_ausm_plus_higher_order.py:66: RuntimeWarning: invalid value encountered in divide
  r_1 = delta_half1[:,i] / delta_half1[:,i-1] # Eqn 66 r_j
c:\Gitlab\shock-tube\ausm_higher_order\main_ausm_plus_higher_order.py:67: RuntimeWarning: invalid value encountered in divide
  r_2 = delta_half1[:,i] / delta_half1[:,i+1] # Eqn 66 r_j+1
c:\Gitlab\shock-tube\ausm_higher_order\main_ausm_plus_higher_order.py:66: RuntimeWarning: divide by zero encountered in divide
  r_1 = delta_half1[:,i] / delta_half1[:,i-1] # Eqn 66 r_j
c:\Gitlab\shock-tube\ausm_higher_order\main_ausm_plus_higher_order.py:67: RuntimeWarning: divide by zero encountered in divide
  r_2 = delta_half1[:,i] / delta_half1[:,i+1] # Eqn 66 r_j+1
c:\Gitlab\shock-tube\ausm_higher_order\main_ausm_plus_higher_order.py:66: RuntimeWarning: invalid value encountered in divide
  r_1 = delta_half1[:,i] / delta_half

Moviepy - Building video ausm+_higher_order_results.mp4.
Moviepy - Writing video ausm+_higher_order_results.mp4



Moviepy - Done !
Moviepy - video ready ausm+_higher_order_results.mp4
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
